In [ ]:
import sqlite3
import pandas as pd
import originpro as op
import os
import sys
import numpy as np # 引入 numpy 处理除以零的情况

# ==========================================
# 配置区域
# ==========================================
db_path = "IV.db"
target_indices = [3, 4, 5, 6, 7, 8]  

# ==========================================
# 辅助函数
# ==========================================
def hex_to_origin_color(hex_str):
    hex_str = hex_str.lstrip('#')
    r = int(hex_str[0:2], 16)
    g = int(hex_str[2:4], 16)
    b = int(hex_str[4:6], 16)
    return r + (g * 256) + (b * 65536)

def style_graph_auto_fix(layer, color_hex, x_title, y_title):
    """
    通用样式函数：
    1. 设置线条/散点颜色
    2. 隐藏 Top/Right 坐标轴
    3. 强制修复标题 (防止上划线/乱码)
    """
    target_color = hex_to_origin_color(color_hex)
    
    # 1. 设置绘图样式
    plot_list = layer.plot_list()
    if plot_list:
        plot = plot_list[0]
        plot.set_int("symbol.shape", 1)       
        plot.set_float("symbol.size", 3)     
        plot.set_int("symbol.color", target_color)
        plot.set_int("symbol.fillcolor", target_color)
        plot.set_int("line.color", target_color)
        plot.set_float("line.width", 1)
    
    # 2. 坐标轴框架美化 & 标题强制重写
    layer.rescale()
    
    # LabTalk 脚本构建
    # 使用 Python 的 f-string 将标题传入 LabTalk
    # 注意：LabTalk 字符串需要用双引号，所以 Python 中外层用单引号
    cmds = [
        # --- 隐藏多余刻度 ---
        "layer.x.top.show=1;", "layer.x.top.label.show=0;", "layer.x.top.title.show=0;", 
        "layer.x.top.ticks.major=0;", "layer.x.top.ticks.minor=0;",
        "layer.y.right.show=1;", "layer.y.right.label.show=0;", "layer.y.right.title.show=0;", 
        "layer.y.right.ticks.major=0;", "layer.y.right.ticks.minor=0;",
           ]
    
    for cmd in cmds:
        layer.lt_exec(cmd)

# ==========================================
# 主程序
# ==========================================

if not os.path.exists(db_path):
    print(f"错误：找不到数据库 {db_path}")
    sys.exit(1)

if op.oext:
    op.set_show(True)

conn = sqlite3.connect(db_path)

for idx in target_indices:
    table_name = f"results-1-{idx}"
    print(f"\n>>> 正在处理表格: {table_name} ...")
    
    # ---------------------------
    # A. 读取数据 (Pandas)
    # ---------------------------
    try:
        df = pd.read_sql(f"SELECT volt, curr FROM '{table_name}';", conn)
    except Exception as e:
        print(f"   [跳过] 无法读取表 {table_name}: {e}")
        continue 

    if df.empty: continue
    df = df.dropna(subset=["volt", "curr"])
    
    # ---------------------------
    # B. 数据计算 (计算电阻)
    # ---------------------------
    # 1. 过滤掉电流为 0 的点，防止除以零报错
    df = df[df["curr"] != 0]
    
    df_origin = df.copy()
    
    # 转换电流单位 (A -> uA)
    df_origin["curr_uA"] = df_origin["curr"] * 1e6 
    
    # 计算电阻 (Ohm) = V / I (使用原始 Ampere 计算)
    # 这里的 abs 是可选的，取决于你是否需要负电阻显示。通常取绝对值看幅值，或者保留符号。
    # 这里保留符号：R = V/I
    df_origin["resistance"] = df_origin["volt"] / df_origin["curr"]

    # ---------------------------
    # C. 导出 Excel
    # ---------------------------
    excel_filename = f"Ready_IV_IR_{idx}.xlsx"
    df_origin.to_excel(excel_filename, index=False)
    full_path = os.path.abspath(excel_filename)
    print(f"   数据已准备 (含电阻): {excel_filename}")

    # ---------------------------
    # D. Origin 数据导入
    # ---------------------------
    print(f"   正在 Origin 中处理数据...")
    
    # 创建工作表
    wks = op.new_sheet()
    wks.name = f"Data_{idx}"
    wks.from_file(full_path, keep_DC=False)
    
    # 设置列属性 (0:Volt, 1:Curr_raw(可能被跳过), 2:Curr_uA, 3:Resistance)
    # pandas导出顺序: volt, curr, curr_uA, resistance
    # Origin列索引:   0     1     2        3
    
    # 显式指定 XY 属性
    wks.cols_axis('xyyy') # 假设 col0 是 X，后面都是 Y (稍后绘图指定具体列)

    # 设置标签 (仅作参考，实际标题由 style 函数控制)
    wks.set_label(0, "V") 
    wks.set_label(2, "I (uA)") 
    wks.set_label(3, "R (Ω)") 

    # ---------------------------
    # E. 绘制图表 1: IV 曲线
    # ---------------------------
    # I-V 曲线通常是 Current(Y) vs Voltage(X) ? 
    # 原代码逻辑：plot(wks, 1, 0) -> Col 1 vs Col 0. 
    # 现在 Col 2 是 uA。我们要画 Col 2 (Y) vs Col 0 (X)。
    
    graph_iv = op.new_graph(template='linesymb')
    graph_iv.name = f"Graph_IV_{idx}" # 尝试重命名
    
    lay_iv = graph_iv[0]
    # 绘制：Y = Col(2)[curr_uA], X = Col(0)[volt]
    lay_iv.add_plot(wks, 2, 0) 
    
    # 应用样式 (IV) - 蓝色系
    style_graph_auto_fix(lay_iv, "#8E9CFF", "Voltage (V)", "Current (\\g(m)A)")
    
    # ---------------------------
    # F. 绘制图表 2: IR 曲线
    # ---------------------------
    # IR 曲线通常是 Resistance(Y) vs Current(X)
    
    graph_ir = op.new_graph(template='linesymb')
    graph_ir.name = f"Graph_IR_{idx}"
    
    lay_ir = graph_ir[0]
    # 绘制：Y = Col(3)[resistance], X = Col(2)[curr_uA]
    lay_ir.add_plot(wks, 3, 2) 
    
    # 应用样式 (IR) - 橙色系，并使用 \g(W) 显示欧姆
    style_graph_auto_fix(lay_ir, "#FFA500", "Current (\\g(m)A)", "Resistance (\\g(W))")
    
    # 整理窗口
    op.lt_exec("win -s T;") 

conn.close()
print("\n>>> 所有 IV & IR 图表处理完成！")

if op.oext:
    input(">>> 按 [回车键] 结束脚本连接...")